In [60]:
# missionaries and Cannibals Problem - setup
from collections import defaultdict, deque, Counter
import heapq
import time

class Problem(object):
    def __init__(self,initial_state):
        self.initial_state = initial_state
    def actions(self,state):
        return state.expand()
    def goal(self, state):
        return state.goal()
    def h(self, state):
        return state.heuristic()

    

class Node:
    def __init__(self, missionaries, cannibals, boat, parent=None, path_cost=0):
        self.m = missionaries
        self.c = cannibals
        self.b = boat
        self.parent = parent
        self.path_cost = path_cost
        
    
    # Print a good looking representation
    def __repr__(self):
        return f"({self.m},{self.c},{self.b}) |||| ({3 - self.m},{3 - self.c},{1 - self.b})"
    # compare cost
    def __lt__(self,comp):
        return self.path_cost + self.heuristic() < comp.path_cost + comp.heuristic()
    # get the state of the node
    def state(self):
        return (self.m, self.c, self.b)

    # check valid node
    def valid_state(self):
        if (self.m < 0 or self.m > 3 or self.c < 0 or self.c > 3): # correct number of Each
            return False
        if (self.m > 0 and self.m < self.c): # Cannibals > missionaries before crossing
            return False
        if (3 - self.m > 0 and (3- self.m) < (3- self.c)): # Cannibals > missionaries on other side
            return False
        return True
    # check for goal state
    def goal(self):
        return (self.m, self.c, self.b) == (0,0,0)
    # exapnd node by going through possible moves
    def expand(self):
        children = []
        moves = [(2, 0), (1, 0), (0, 1), (0, 2), (1, 1)] 
        for m, c in moves:
            if (self.b == 1): # Crossing
                child_m = self.m - m
                child_c = self.c - c
                child_b = 0
            else: # returning 
                child_m = self.m + m
                child_c = self.c + c
                child_b = 1

            child_cost = self.path_cost + 1
            child = Node(child_m, child_c, child_b, path_cost=child_cost, parent=self)

            if (child.valid_state()):
                children.append(child)
        return children
    # return path for solution readability
    def get_path(self):
        path = []
        current_node = self
        while current_node:
            path.append(current_node)
            current_node = current_node.parent
        return list(reversed(path))  # reverse order
    def path_cost(self, c=0):
        return c+1
    # Heuristic for a star - how many people on left side divided by 2
    def heuristic(self):
        return (self.m + self.c)/2
    
def BFS(problem):
    root = problem.initial_state # initial node
    if(root.goal()): 
        return root.get_path()
    # create frontier and reached set
    frontier = deque([root])
    reached = set([root.state()])
    while frontier: # isnt empty
        node = frontier.popleft() 
        for child in node.expand(): # get all possible valid children, check for goal state
            if(child.goal()):
                return child.get_path()
            reached.add(child.state())
            frontier.append(child)
    return None

# A Star search implementation with best first
def best_first(problem, f):
    root = (problem.initial_state)
    frontier = []
    heapq.heappush(frontier, (f(root),root))
    reached = set()
    while frontier:
        _, node = heapq.heappop(frontier)
        if (problem.goal(node)):
            return node.get_path()
        reached.add(node)
        for child in problem.actions(node):
            state = child.state()
            if (state not in reached):
                heapq.heappush(frontier, (f(child), child))

def a_star(problem):
    return best_first(problem, f=lambda n: n.path_cost + n.heuristic())

init = Node(3,3,1)
problem = Problem(init)

start_BFS = time.time()
solution_BFS = BFS(problem)
end_BFS = time.time()

if solution_BFS:
    print("BFS Solution:")
    path_cost = 0
    # print path
    for step in solution_BFS:
        print(step)
        path_cost += 1
    # print path cost
    result =f'path cost: {path_cost - 1}'
    print(result)
    #print time
    BFS_time = f"BFS time taken: {end_BFS - start_BFS:.4f} seconds"
    print(BFS_time)
else:
    print("No solution exists.")


start_astar = time.time()
solution_astar = a_star(problem)
end_astar = time.time()
if solution_astar:
    print("A* Solution:")
    path_cost = 0
    # print path
    for step in solution_astar:
        print(step)
        path_cost += 1
    # print path cost
    result =f'path cost: {path_cost - 1}'
    print(result)
    #print time
    a_star_time = f"A* time taken: {end_astar - start_astar:.4f} seconds"
    print(BFS_time)
else:
    print("No solution exists.")



BFS Solution:
(3,3,1) |||| (0,0,0)
(3,1,0) |||| (0,2,1)
(3,2,1) |||| (0,1,0)
(3,0,0) |||| (0,3,1)
(3,1,1) |||| (0,2,0)
(1,1,0) |||| (2,2,1)
(2,2,1) |||| (1,1,0)
(0,2,0) |||| (3,1,1)
(0,3,1) |||| (3,0,0)
(0,1,0) |||| (3,2,1)
(1,1,1) |||| (2,2,0)
(0,0,0) |||| (3,3,1)
path cost: 11
BFS time taken: 0.0170 seconds
A* Solution:
(3,3,1) |||| (0,0,0)
(2,2,0) |||| (1,1,1)
(3,2,1) |||| (0,1,0)
(3,0,0) |||| (0,3,1)
(3,1,1) |||| (0,2,0)
(1,1,0) |||| (2,2,1)
(2,2,1) |||| (1,1,0)
(0,2,0) |||| (3,1,1)
(0,3,1) |||| (3,0,0)
(0,1,0) |||| (3,2,1)
(1,1,1) |||| (2,2,0)
(0,0,0) |||| (3,3,1)
path cost: 11
BFS time taken: 0.0170 seconds
